In [29]:
#libraries
'''
!pip install transformers==4.31.0 
!pip install sentence-transformers==2.2.2 
!pip accelerate==0.21.0 \
!pip xformers==0.0.20 \
'''

  Obtaining dependency information for transformers==4.31.0 from https://files.pythonhosted.org/packages/21/02/ae8e595f45b6c8edee07913892b3b41f5f5f273962ad98851dc6a564bbb9/transformers-4.31.0-py3-none-any.whl.metadata
  Using cached transformers-4.31.0-py3-none-any.whl.metadata (116 kB)
  Obtaining dependency information for tokenizers!=0.11.3,<0.14,>=0.11.1 from https://files.pythonhosted.org/packages/62/41/93d3135ec30f596a71490ce11a73572190fe80e85a2aea18f116a520cc41/tokenizers-0.13.3-cp311-cp311-win_amd64.whl.metadata
  Using cached tokenizers-0.13.3-cp311-cp311-win_amd64.whl.metadata (6.9 kB)
Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
Using cached tokenizers-0.13.3-cp311-cp311-win_amd64.whl (3.5 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninsta

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.1.0 requires transformers<5.0.0,>=4.38.0, but you have transformers 4.31.0 which is incompatible.


  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.1.0
    Uninstalling sentence-transformers-3.1.0:
      Successfully uninstalled sentence-transformers-3.1.0


ERROR: unknown command "accelerate==0.21.0"



In [30]:
#specific version to avoid error with vector store
#!pip install chromadb==0.5.3
#!pip install datasets

In [22]:
#loading training data for providing context to the model
import pandas as pd
train_data_df=pd.read_csv("C:/Users/USER/Documents/Projects/Instruction Dataset/LLM Training Data/train_data_split.csv")
## Convert the DataFrame to a format compatible with Hugging Face Datasets
train_data_df['Instruction'] = train_data_df['Instruction']+ ". Input Text: " + train_data_df['input_text'] + "\n\nExtracted Properties:\n\n"+ train_data_df['Output'].astype(str)
train_dataset_dict = {
    'instruction': train_data_df['Instruction'].tolist()
}

In [23]:
#verify
#train_dataset_dict

In [24]:
#creating context document from the list
docs=train_dataset_dict['instruction']

In [31]:
#initializing the embedding model
from sentence_transformers import SentenceTransformer
import chromadb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
import torch
embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', cache_folder = '/data/base_models')

In [ ]:
#do the embedding
embeddings = embedding_model.encode(docs)

In [ ]:
#verify the embeddings
embeddings.shape

In [ ]:
#indexing-run this only once
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="q-rag_llama")

In [ ]:
#addding the train data
collection.add(
    embeddings = embeddings,
    documents=docs,
    ids= [str(i) for i in range(len(docs))]
)

In [ ]:
#retrieving
def retrieve_vector_db(query, n_results=3):
    results = collection.query(
    query_embeddings = embedding_model.encode(query).tolist(),
    n_results=n_results
    )
    return results['documents']

In [ ]:
#loading test data for model evaluation
import pandas as pd
test_data_path="C:/Users/USER/Documents/Projects/Instruction Dataset/LLM Training Data/test_data_split.csv"

In [ ]:
#processing the test data to add the queries
#defining the queries for the properties-we need to choose a query based on the input
query_temperature="working temperature of the quantum cascade laser"
query_power= "optical power of the quantum cascade laser"
query_frequency="lasing frequency of the quantum cascade laser"
query_hs="heterostructure of the quantum cascade laser"
query_design_type="design type of the quantum cascade laser"

In [ ]:
#checking for the query keywords
def check_text(row, column_name):
    text = row[column_name].lower()
    if "working temperature" in text:
        return query_temperature
    elif "optical power" in text:
        return query_power
    elif "lasing frequency" in text:
        return query_frequency
    elif "query_hs" in text:
        return query_hs
    else:
        return query_design_type

In [ ]:
#processing the data
def process_csv(input_filepath, output_filepath, column_name):
    # Load the CSV file
    df = pd.read_csv(input_filepath)

    # Check if the specified column exists
    if column_name in df.columns:
        # Apply the check_text function to the specified column and create a new column with the results
        df['queries'] = df.apply(check_text, axis=1, args=(column_name,))
    else:
        print(f"Column '{column_name}' does not exist in the CSV file.")
        return

    # Save the modified DataFrame to a new CSV file
    df.to_csv(output_filepath, index=False)
    print(f"Processed file saved as '{output_filepath}'.")

In [ ]:
#processing the test data
input_csv = test_data_path
output_csv = 'processed_test_data.csv'
column_to_check = 'Instruction'
process_csv(input_csv, output_csv, column_to_check)

In [ ]:
#loading the processed test data
test_data_df=pd.read_csv('processed_test_data.csv')

In [ ]:
#putting the data in some order and renaming the columns
test_data_df=test_data_df[['queries', 'Instruction', 'input_text']]
test_data_df.rename(columns={'Instruction': 'instructions', 'input_text': 'texts'}, inplace=True)

In [ ]:
#verifying the ordered data
#test_data_df

In [ ]:
#we need to generate a list of prompts will be passed to the model.
prompts=[]
for a,b,c in zip(test_data_df['queries'],test_data_df['instructions'],test_data_df['texts']):
    results=retrieve_vector_db(a)
    context=results[0]
    prompt = f'''
[INST]
Problem Definition: Extraction of quantum cascade laser properties from text entails extracting properties from a given text description. This should be done without providing any other additional information or explanations. The output format should correspond to the one in the example sentences. Example sentences containing an instruction, input text and the extracted properties are given below:

Example Sentences: {context}

Instruction: {b}

Input Text: {c}

[/INST]
'''
    prompts.append(prompt)

In [ ]:
#verifying the list of prompts generated for each row in the dataset
#display the prompt
#prompts

In [22]:
#verify the length of the prompt list
print(len(prompts))

130


In [23]:
#GPT
import openai
import os
from IPython.display import Markdown
api_key="OPEN_API_KEY"

In [24]:
import openai

# Set your API key securely
openai.api_key = api_key

def chatWithGPT4(user_text, print_output=False):
    try:
        # Creating text completions using the updated `ChatCompletion` class
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",  # Easily switch between "gpt-3.5-turbo-0613", "gpt-4", "text-davinci-004", or others
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": user_text}],
            max_tokens=3000
        )
        text_output = response['choices'][0]['message']['content'] if response['choices'] else "No response generated."
        if print_output:
            print(text_output)
        return text_output
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [25]:
#checking a sample prompt at a particular index in the list of prompts
prompts[0]

"\n[INST]\nProblem Definition: Extraction of quantum cascade laser properties from text entails extracting properties from a given text description. This should be done without providing any other additional information or explanations. The output format should correspond to the one in the example sentences. Example sentences containing an instruction, input text and the extracted properties are given below:\n\nExample Sentences: ['From the following sentence, please extract the lasing frequency of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text. Input Text: We noticed a shift towards higher frequencies from the middle to the outskirts of the wafer in our observation of three GaAs/AlAs heterostructure-based lasers with target frequencies of 3.36 and 3.92 THz.\\t\\t\\n\\nExtracted Properties:\\n\\n3.36 and 3.92 THz.', 'From the following sentence, please extract the lasing frequency of the Quantum Cascade Semiconductor laser devic

In [26]:
#getting the model responses in a list
responses=[]
for prompt in prompts:
    result=chatWithGPT4(prompt)
    responses.append(result)

In [27]:
#dispalying the responses
responses

['Extracted Properties:\n\n3.3 THz',
 'Extracted Properties:\n\n10 mW',
 'Extracted Properties:\n\n2.07 THz, 2.02 THz',
 'Extracted Properties:\n\n190 K, 125 K',
 'Extracted Properties:\n\n107 K, 91 K',
 'Extracted Properties:\n\nGaAs/Al0.23Ga0.77As',
 'Extracted Properties:\n\nIn0.6Ga0.4As/InAs',
 'Extracted Properties:\n\n3.0 THz',
 'Extracted Properties:\n\n185 K',
 'Extracted Properties:\n\n325 K',
 'Extracted Properties:\n\nnone',
 'Extracted Properties:\n\n195 K',
 'Extracted Properties:\n\n24 mW',
 'Extracted Properties:\n\nnone',
 'Extracted Properties:\n\n150 K, 110 K',
 'Extracted Properties:\n\nnone',
 'Extracted Properties:\n\n113 K, 89 K',
 'Extracted Properties:\n\n70K',
 'Extracted Properties:\n\nnone',
 'Extracted Properties:\n\nnone',
 'Extracted Properties:\n\n182 K',
 'Extracted Properties:\n\nnone',
 'Extracted Properties:\n\nGaAs/Al0.15Ga0.85As',
 'Extracted Properties:\n\nnone',
 'Extracted Properties:\n\nLO-phonon-based depopulation',
 'Extracted Properties:\n\n3

In [28]:
#writing the model responses to a csv file
import csv

# Name of the csv file to save the list
file_name = "qcl-RAG-GPT4-Turbo-responses.csv"

# Writing the list to a csv file with one column
with open(file_name, "w", newline="") as file:
    writer = csv.writer(file)
    for item in responses:
        writer.writerow([item])

print("List saved to", file_name)

List saved to qcl-RAG-GPT4-Turbo-responses.csv
